# Tutorial

Let's start with a simple usage:

```
px <expr>
```

Here, `px` starts a Python process and prints the result of evaluating `expr`. For example:

In [1]:
px "3*6"

18


: 1

If the output is a list, it will be printed out in lines:

In [2]:
px "list(range(4))"

0
1
2
3


: 1

The second usage form is as follows:
```
px <expr> <file>
```
In this case, `px` reads `file` before evaluating `expr`. The file will be stored as a *list of lines*, with variable name `x`.

For example, the following command prints the first 2 lines of `example.txt`:

In [3]:
px "x[0:2]" example.txt

1
2


: 1

One could apply a function to lines of a file:

In [4]:
px "['number: ' + u for u in x]" example.txt

number: 1
number: 2
number: 3
number: 4
number: 5


: 1

Since this use case is common, there is an option `-L`. With `-L` option, `px` will run `expr` on each line of input file. Therefore, when using `-L`, variable `x` will correspond to a *single line*, not the entire list of lines.

The last example can be simplified with `-L` as follows:

In [5]:
px -L "'number: ' + x" example.txt

number: 1
number: 2
number: 3
number: 4
number: 5


: 1

One could do math on lines, but note that `px` reads lines as text. So text to int (or float) conversion will be required.

For example:

In [6]:
px -L "int(x)**2" example.txt

1
4
9
16
25


: 1

To achieve the task above, it is also possible to use the `--pre` option which applies a preprocessing function to all lines of input:

In [7]:
px --pre=int -L "x**2" example.txt

1
4
9
16
25


: 1

Also, there are two ways to feed output of another command to `px`:

In [8]:
cat example.txt | px -L "'item: ' + x" -

item: 1l
item: 2
item: 3
item: 4
item: 5


: 1

or:

In [9]:
px -L "'item ' + x" <(cat example.txt)

item 14l
item 2
item 3
item 4
item 5


: 1

The last important usage is where `px` gets multiple files:
```
px <expr> <file_1> <file_2> ... <file_n>
```

In this case, each file will be stored as variables `x`, `y`, `z`, `a`, `b`, ... Just like the case with a single file, using `-L` option will make this variables to store individual lines, instead of list of lines.

For example, assume we want to compute line-wise sum of two files. This command will do the job:

In [10]:
px --pre=int -L "x+y" a.txt b.txt

15
8
10
5
16


: 1

# Examples

*All cells are bash terminals.*

## Generate random numbers

For demonstration purposes, let's first generate a list of random numbers. Traditionally:

In [1]:
cat /dev/null > a.txt # Create new file or clear existing one
for run in {1..5}; do echo $((${RANDOM}%10+1)) >> a.txt; done # Push random numbers to the file

: 1

Or using `px`:

In [12]:
px -i random '[randint(1, 10) for i in range(5)]' > b.txt

: 1

For a list of *unique* random numbers:

In [13]:
px -i random 'sample(range(10), 5)' > b.txt

: 1

## Calculate sum of a series

Traditionally:

In [14]:
awk '{ sum += $1 } END { print sum }' a.txt

32


: 1

Or:

In [15]:
paste -sd+ a.txt | bc

32


: 1

With `px`:

In [16]:
px --pre=int 'sum(x)' a.txt

32


: 1

## Calculate row-wise sum of two series:

With `px`:

In [17]:
px --pre=int '[X+Y for X,Y in zip(x,y)]' a.txt b.txt

3
17
11
5
18


: 1

or with `-L` shorthand:

In [18]:
px --pre=int -L 'x+y' a.txt b.txt

3
17
11
5
18


: 1

## Get last n lines of a file

Traditionally:

In [19]:
tail a.txt -n3

8
3
9


: 1

With `px`:

In [20]:
px 'x[-3:]' a.txt

8
3
9


: 1

## Get unique values in a list

In [21]:
cat a.txt | sort | uniq

3
8
9


: 1

In [22]:
px 'set(x)' a.txt

8
9
3


: 1

## Get number of values in a list

In [23]:
cat a.txt | wc -l

5


: 1

In [24]:
px 'len(x)' a.txt

5


: 1

## Shuffle a list

In [25]:
px -i random 'sample(x, len(x))' a.txt

8
3
9
9
3


: 1

## Reverse a list

In [26]:
px 'x[::-1]' a.txt

9
3
8
9
3


: 1

## Get x from a list where x > 5

In [27]:
px --pre=int 'filter(lambda u: u > 5, x)' a.txt

9
8
9


: 1

## Format series

In [28]:
paste -sd',' a.txt

3,9,8,3,9


: 1

In [29]:
px "','.join(x)" a.txt

3,9,8,3,9


: 1

## Format two series in a table:

In [30]:
paste a.txt b.txt

3	0
9	8
8	3
3	2
9	9


: 1

In [31]:
px -L "f'{x}\t{y}'" a.txt b.txt

3	0
9	8
8	3
3	2
9	9


: 1

## Visualize using `matplotlib`

Plot and show:

In [35]:
px --pre=int -i numpy,matplotlib.pyplot "hist(x)" a.txt -x "show()" 2> /dev/null

[2. 0. 0. 0. 0. 0. 0. 0. 1. 2.]
[3.  3.6 4.2 4.8 5.4 6.  6.6 7.2 7.8 8.4 9. ]
<BarContainer object of 10 artists>
None


: 1

Plot and save:

In [37]:
px --pre=int -i numpy,matplotlib.pyplot "hist(x)" a.txt -x "savefig('a.png')" 2> /dev/null
xdg-open a.png

[2. 0. 0. 0. 0. 0. 0. 0. 1. 2.]
[3.  3.6 4.2 4.8 5.4 6.  6.6 7.2 7.8 8.4 9. ]
<BarContainer object of 10 artists>
None


: 1

## Select random files in a directory

In [42]:
px -i random 'sample(x, 5)' <(ls /dev/tty*)

/dev/tty27
/dev/ttyS22
/dev/ttyS5
/dev/tty35
/dev/tty14


: 1

## Calculate total duration of music files in a path

In [43]:
ls ~/Music/Best/Rock/*.mp3 | \
    xargs -I ss ffmpeg -i ss 2>&1 | \
    grep -Po "Duration:\s\K(\d|:)+" | \
    px -L -i pytimeparse.timeparse,datetime "timeparse(x)" - --post "lambda u: timedelta(seconds=sum(u))"

3:08:27h


: 1

# Quick for loop

In [11]:
px 'range(10)' | xargs -Ix printf 'Hi! This is iteration #%d.\n' x

Hi! This is iteration #0.
Hi! This is iteration #1.
Hi! This is iteration #2.
Hi! This is iteration #3.
Hi! This is iteration #4.
Hi! This is iteration #5.
Hi! This is iteration #6.
Hi! This is iteration #7.
Hi! This is iteration #8.
Hi! This is iteration #9.


: 1